In [1]:
import torch
from models import model_dict
import os

from dataset import IrisDataset
from dataset import transform
from torch.utils.data import DataLoader 

import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from utils import get_predictions

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# arguments
useGPU = "cuda" # or cpu
modelname = 'densenet'
filename='./best_model.pkl'
bs = 4 # batch size
save = 'test10' # path

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
# setup model with GPU
model = model_dict[modelname]
device= torch.device(useGPU)
# Load the model from the .pkl file
model.load_state_dict(torch.load(filename))
model = model.to(device)
# evaluate model
model.eval()

DenseNet2D(
  (down_block1): DenseNet2D_down_block(
    (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv21): Conv2d(33, 32, kernel_size=(1, 1), stride=(1, 1))
    (conv22): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv31): Conv2d(65, 32, kernel_size=(1, 1), stride=(1, 1))
    (conv32): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (max_pool): AvgPool2d(kernel_size=None, stride=None, padding=0)
    (relu): LeakyReLU(negative_slope=0.01)
    (dropout1): Dropout(p=0.2, inplace=False)
    (dropout2): Dropout(p=0.2, inplace=False)
    (dropout3): Dropout(p=0.2, inplace=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (down_block2): DenseNet2D_down_block(
    (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv21): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
    (conv22): Conv2d(32, 32, kernel_size=(3, 3), stride=

In [5]:
print(type(model))

INPUT_XDIM = 400
INPUT_YDIM = 640

# bs , number of channels, xdim,
x = torch.randn(bs, 1, INPUT_XDIM, INPUT_YDIM, requires_grad=False).to(device)
model_outputfile = "best_model.onnx"
torch.onnx.export(model,               # model being run
                x,                         # model input (or a tuple for multiple inputs)
                model_outputfile,   # where to save the model (can be a file or file-like object)
                export_params=True,        # store the trained parameter weights inside the model file
                opset_version=12,          # the ONNX version to export the model to
                # do_constant_folding=True,  # whether to execute constant folding for optimization
                do_constant_folding=False,  # whether to execute constant folding for optimization
                input_names = ['input'],   # the model's input names
                output_names = ['output'], # the model's output name
)

# ===================== from rose =====================
# reference: https://github.com/ucb-bar/RoSE/blob/a743406eed87abe10dced40f141e9739a8978f52/env/train/train_resnet.py#L245C1-L257C6
#     x = torch.randn(1, 3, INPUT_DIM, INPUT_DIM, requires_grad=True).to(device)
# outputfile = f"./env/train/trail_dnn_{dnn_name}.onnx"
# torch.onnx.export(model,               # model being run
#                 x,                         # model input (or a tuple for multiple inputs)
#                 outputfile,   # where to save the model (can be a file or file-like object)
#                 export_params=True,        # store the trained parameter weights inside the model file
#                 opset_version=12,          # the ONNX version to export the model to
#                 # do_constant_folding=True,  # whether to execute constant folding for optimization
#                 do_constant_folding=False,  # whether to execute constant folding for optimization
#                 input_names = ['input'],   # the model's input names
#                 output_names = ['output'], # the model's output name
# )

<class 'densenet.DenseNet2D'>


In [8]:
# ONNX attempt
# onnx documentation: https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html
import onnx
import onnxruntime as ort

ort_sess = ort.InferenceSession(model_outputfile) # providers = 'CUDAExecutionProvider'
onnx_model = onnx.load(model_outputfile)

# x = torch.randn(bs, 1, INPUT_XDIM, INPUT_YDIM, requires_grad=False).to(device)
input = np.random.random((bs, 1, INPUT_XDIM, INPUT_YDIM)).astype(np.float32)

outputs = ort_sess.run(None, {'input': input})

In [20]:
os.makedirs('test/labels/',exist_ok=True)
os.makedirs('test/mask/',exist_ok=True)
# step1: preprocess image
# IrisDataset is preprocessing for test inputs
test_set = IrisDataset(filepath = 'Semantic_Segmentation_Dataset/',\
                            split = 'test',transform = transform)
# create iterable from preprocessed test inputs  
testloader = DataLoader(test_set, batch_size = bs,
                        shuffle=False, num_workers=2)
# step2
for i, batchdata in tqdm(enumerate(testloader),total=len(testloader)):
    # batchdata based on get function of IrisDataset
    img,labels,index,x,y= batchdata
    # ========== model inference ===========
    # data = img.to(device)       
    # output = model(data)
    in1 = np.random.random((bs, 1, INPUT_XDIM, INPUT_YDIM)).astype(np.float32)
    input = img.numpy()
    print(in1.shape, img.size())
    outputs = ort_sess.run(None, {'input': input})
    # parse model output 
    o = outputs[0]
    print(o.shape, type(o))
    o_tensor = torch.from_numpy(o)
    predict = get_predictions(o_tensor)
    # ======================================
    for j in range (len(index)):       
        pred_img = predict[j].cpu().numpy()/3.0
        inp = img[j].squeeze() * 0.5 + 0.5
        img_orig = np.clip(inp,0,1)
        img_orig = np.array(img_orig)
        combine = np.hstack([img_orig,pred_img])
        plt.imsave('test/mask/{}.jpg'.format(index[j]),combine)

  0%|          | 0/1 [00:00<?, ?it/s]

(4, 1, 400, 640) torch.Size([2, 1, 400, 640])


InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : Got invalid dimensions for input: input for the following indices
 index: 0 Got: 2 Expected: 4
 Please fix either the inputs/outputs or the model.

In [ ]:
# onnx ~
# IrisDataset is preprocessing for test inputs
test_set = IrisDataset(filepath = 'Semantic_Segmentation_Dataset/',\
                            split = 'test',transform = transform)
# create iterable from preprocessed test inputs  
testloader = DataLoader(test_set, batch_size = bs,
                        shuffle=False, num_workers=2)

os.makedirs('test1/labels/',exist_ok=True)
os.makedirs('test1/mask/',exist_ok=True)
# disable computing gradients
count = 0
with torch.no_grad():
    # tqdm for pretty progress output lol
    for i, batchdata in tqdm(enumerate(testloader),total=len(testloader)):
        # batchdata based on get function of IrisDataset
        img,labels,index,x,y= batchdata
        print(i, index)
        print(img.size(), torch.max(img), torch.min(img))

        # ========== model inference ===========
        data = img.to(device)       
        output = model(data)
        # parse model output 
        predict = get_predictions(output)
        # ======================================

        # post-processing
        for j in range (len(index)):       
            pred_img = predict[j].cpu().numpy()/3.0
            inp = img[j].squeeze() * 0.5 + 0.5
            img_orig = np.clip(inp,0,1)
            img_orig = np.array(img_orig)
            combine = np.hstack([img_orig,pred_img])
            plt.imsave('test/mask/{}.jpg'.format(index[j]),combine)
        count = count + 1
        if (count == 20):
            break

In [74]:
torch_out = model(x)
# print(f"onnx outputs: {outputs}")
print(f"torch outputs: {torch_out}")

torch outputs: tensor([[[[-6.2462e+00, -1.5522e+01, -2.0919e+01,  ..., -3.8146e+01,
           -3.3494e+01, -1.9651e+01],
          [-8.3552e+00, -2.1258e+01, -2.6980e+01,  ..., -4.8044e+01,
           -4.3221e+01, -2.9118e+01],
          [-2.6113e+00, -9.0946e+00, -1.2910e+01,  ..., -1.6944e+01,
           -1.6979e+01, -1.0541e+01],
          ...,
          [-4.0616e-01, -1.6885e+00,  9.0543e+00,  ..., -2.5813e+00,
           -6.0433e+00, -5.0296e+00],
          [-2.5671e+00, -6.6061e+00,  9.7945e+01,  ..., -2.8505e+00,
           -6.2723e+00, -5.5906e+00],
          [ 1.4756e+01,  2.5771e+01,  3.2860e+01,  ...,  7.5446e+00,
           -3.5225e+00, -3.2198e+00]],

         [[-2.5345e+01, -4.0968e+01, -5.4577e+01,  ..., -8.7417e+01,
           -7.2479e+01, -4.0010e+01],
          [-2.3187e+01, -3.7284e+01, -5.0853e+01,  ..., -8.1502e+01,
           -6.7638e+01, -4.1710e+01],
          [-1.6601e+01, -2.4373e+01, -3.2193e+01,  ..., -4.5194e+01,
           -3.9348e+01, -2.4767e+01],
     

In [72]:
# for j in range (len(4)):     
#     predict = get_predictions(output)  
#     pred_img = predict[j].cpu().numpy()/3.0
#     inp = img[j].squeeze() * 0.5 + 0.5
#     img_orig = np.clip(inp,0,1)
#     img_orig = np.array(img_orig)
#     combine = np.hstack([img_orig,pred_img])
#     plt.imsave('test/mask/{}.jpg'.format(index[j]),combine)

[array([[[[-2.83720326e+00, -7.38337851e+00, -8.35018444e+00, ...,
          -1.96863804e+01, -1.56755724e+01, -4.17669249e+00],
         [-3.73709893e+00, -1.02495880e+01, -1.19802151e+01, ...,
          -2.62000790e+01, -2.06824131e+01, -1.22247839e+01],
         [-7.03891814e-01, -4.53805590e+00, -4.19158506e+00, ...,
          -9.13104630e+00, -6.79668045e+00, -3.08595419e+00],
         ...,
         [ 3.08670044e+00, -1.62578392e+00,  1.80466980e-01, ...,
          -1.14232242e+00, -3.68748498e+00, -3.05427933e+00],
         [ 1.48747265e-02, -2.78587747e+00,  9.62786496e-01, ...,
          -1.63332701e+00, -3.73191786e+00, -3.52284598e+00],
         [ 7.01419210e+00,  1.41754227e+01,  1.03761082e+01, ...,
           4.34707785e+00, -2.00746679e+00, -1.77141702e+00]],

        [[-1.20363493e+01, -1.79138336e+01, -2.28193645e+01, ...,
          -4.34818268e+01, -3.28717537e+01, -1.29997301e+01],
         [-1.14044561e+01, -1.72769890e+01, -2.18324356e+01, ...,
          -4.13155785

In [19]:
# onnx ~
# IrisDataset is preprocessing for test inputs
test_set = IrisDataset(filepath = 'Semantic_Segmentation_Dataset/',\
                            split = 'test',transform = transform)
# create iterable from preprocessed test inputs  
testloader = DataLoader(test_set, batch_size = bs,
                        shuffle=False, num_workers=2)

os.makedirs('test1/labels/',exist_ok=True)
os.makedirs('test1/mask/',exist_ok=True)
# disable computing gradients
count = 0
with torch.no_grad():
    # tqdm for pretty progress output lol
    for i, batchdata in tqdm(enumerate(testloader),total=len(testloader)):
        # batchdata based on get function of IrisDataset
        img,labels,index,x,y= batchdata
        print(i, index)
        print(img.size(), torch.max(img), torch.min(img))

        # ========== model inference ===========
        data = img.to(device)       
        output = model(data)
        # parse model output 
        predict = get_predictions(output)
        # ======================================

        # post-processing
        for j in range (len(index)):       
            pred_img = predict[j].cpu().numpy()/3.0
            inp = img[j].squeeze() * 0.5 + 0.5
            img_orig = np.clip(inp,0,1)
            img_orig = np.array(img_orig)
            combine = np.hstack([img_orig,pred_img])
            plt.imsave('test/mask/{}.jpg'.format(index[j]),combine)
        count = count + 1
        if (count == 20):
            break

  0%|          | 0/1 [00:00<?, ?it/s]

0 ('000009', '000010')
torch.Size([2, 1, 400, 640]) tensor(1.) tensor(-0.8902)


FileNotFoundError: [Errno 2] No such file or directory: 'test/mask/000009.jpg'

In [18]:
import shutil
try:
    os.rename('test', save)
except OSError:
    shutil.rmtree(save)
    os.rename('test', save)

In [56]:
# IrisDataset is preprocessing for test inputs

test_set = IrisDataset(filepath = 'Semantic_Segmentation_Dataset/',\
                            split = 'test',transform = transform)
# create iterable from preprocessed test inputs  
testloader = DataLoader(test_set, batch_size = bs,
                        shuffle=False, num_workers=2)

os.makedirs('test/labels/',exist_ok=True)
os.makedirs('test/mask/',exist_ok=True)

In [60]:
# disable computing gradients
count = 0
with torch.no_grad():
    # tqdm for pretty progress output lol
    for i, batchdata in tqdm(enumerate(testloader),total=len(testloader)):
        # batchdata based on get function of IrisDataset
        img,labels,index,x,y= batchdata
        print(i, index)
        print(img.size(), torch.max(img), torch.min(img))

        # ========== model inference ===========
        data = img.to(device)       
        output = model(data)
        # parse model output 
        predict = get_predictions(output)
        # ======================================

        # post-processing
        for j in range (len(index)):       
            pred_img = predict[j].cpu().numpy()/3.0
            inp = img[j].squeeze() * 0.5 + 0.5
            img_orig = np.clip(inp,0,1)
            img_orig = np.array(img_orig)
            combine = np.hstack([img_orig,pred_img])
            plt.imsave('test/mask/{}.jpg'.format(index[j]),combine)
        count = count + 1
        if (count == 20):
            break

 33%|███▎      | 1/3 [00:00<00:00,  4.46it/s]

0 ('000001', '000002', '000003', '000004')
torch.Size([4, 1, 400, 640]) tensor(1.) tensor(-0.9137)
1 ('000005', '000006', '000007', '000008')
torch.Size([4, 1, 400, 640]) tensor(1.) tensor(-0.8902)


100%|██████████| 3/3 [00:00<00:00,  6.66it/s]

2 ('000009', '000010')
torch.Size([2, 1, 400, 640]) tensor(1.) tensor(-0.8902)


In [ ]:
"""
# TODO: break this down
# disable computing gradients
count = 0
with torch.no_grad():
    # tqdm for pretty progress output lol
    for i, batchdata in tqdm(enumerate(testloader),total=len(testloader)):
        # batchdata based on get function of IrisDataset
        img,labels,index,x,y= batchdata
        if count <= 1:
            print('===========types============')
            print('img', type(img)) # tensors
            print('labels', type(labels))
            print('index', type(index), index) # tuple of pic names
            print('x', type(x))
            print('y', type(y))
        data = img.to(device)       
        output = model(data)            
        predict = get_predictions(output)
        for j in range (len(index)):       
            np.save('test/labels/{}.npy'.format(index[j]),predict[j].cpu().numpy())
            try:
                plt.imsave('test/output/{}.jpg'.format(index[j]),255*labels[j].cpu().numpy())
            except:
                pass
            
            pred_img = predict[j].cpu().numpy()/3.0
            inp = img[j].squeeze() * 0.5 + 0.5
            img_orig = np.clip(inp,0,1)
            img_orig = np.array(img_orig)
            combine = np.hstack([img_orig,pred_img])
            plt.imsave('test/mask/{}.jpg'.format(index[j]),combine)
        
        count = count + 1
        if (count == 20):
            break
"""

In [43]:
import shutil
try:
    os.rename('test', save)
except OSError:
    shutil.rmtree(save)
    os.rename('test', save)